# Limpieza y exploración de datos
Por Sergio Rasillo Flores

## Creación del dataset

In [ ]:
# Carga del dataset Titanic desde repositorio de Kaggle
import pandas as pd
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

## Bloque 1: Exploración inicial

In [ ]:
# BLOQUE 1: EXPLORACIÓN INICIAL

# Ejercicio 01. Primera inspección
# Análisis dimensional del dataset y uso de memoria
print("="*70)
print(f"Tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
print(f"\nLos tipos de datos de cada columna son:\n{df.dtypes}")
print(f"\nEl dataframe ocupa {df.memory_usage(deep=True).sum() / 1024:.2f} KB.")
print("="*70 + "\n")

# Ejercicio 02. Estadísticas descriptivas
# Resumen estadístico de variables numéricas y detección de anomalías
print("="*70)
print(df.describe())
print()

edad_media = df["Age"].mean()
print(f"La edad media es de {edad_media:.2f} años.")
tarifa_media = df["Fare"].mean()
print(f"La tarifa media es de {tarifa_media:.2f} libras.")

# Análisis de anomalías
print(f"\nTarifa máxima: {df['Fare'].max():.2f} (posible outlier, 16x la media)")
print(f"Hay {(df['Fare'] == 0).sum()} pasajeros con tarifa 0.00 (sospechoso)")

print("="*70 + "\n")

# Ejercicio 03. Distribución de variables categóricas
# Conteo de frecuencias para Sex, Pclass y Embarked
print("="*70)

# Sexo
print("Distribución por sexo:")
sex_counts = df['Sex'].value_counts()
print(sex_counts)
print(f"Hay {sex_counts['male']} hombres y {sex_counts['female']} mujeres.\n")

# Clase
print("Distribución por clase:")
class_counts = df['Pclass'].value_counts().sort_index()
print(class_counts)
print(f"Primera clase: {class_counts[1]}, Segunda: {class_counts[2]}, Tercera: {class_counts[3]}\n")

# Puerto de embarque
print("Distribución por puerto:")
embarked_counts = df['Embarked'].value_counts()
print(embarked_counts)
print(f"El puerto con más pasajeros fue {embarked_counts.idxmax()} con {embarked_counts.max()} pasajeros")

print("="*70 + "\n")

Tiene 891 filas y 12 columnas.

Los tipos de datos de cada columna son:
PassengerId      int64
Survived         int64
Pclass           int64
Name               str
Sex                str
Age            float64
SibSp            int64
Parch            int64
Ticket             str
Fare           float64
Cabin              str
Embarked           str
dtype: object

El dataframe ocupa 285.61 KB.

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   8

## Bloque 2: Diagnóstico de calidad de datos

In [ ]:
# BLOQUE 2: DIAGNÓSTICO DE CALIDAD DE DATOS

# Ejercicio 04. Mapa de valores faltantes
# Informe completo de missing values ordenado por porcentaje descendente
print("="*70)
informe = pd.DataFrame({
    'Valores_faltantes': df.isna().sum(),
    'Porcentaje': ((df.isna().sum() / len(df)) * 100).round(2),
    'Tipo_dato': df.dtypes
})

informe = informe.sort_values('Porcentaje', ascending=False)
print(informe)
print("="*70 + "\n")

# Ejercicio 05. Análisis de la variable Cabin
# Extracción de cubierta (letra inicial) y análisis de relación con clase
print("="*70)
# Extraer letra inicial (cubierta) sin modificar el df
letra_inicial = df['Cabin'].str[0]

# Ver relación entre cubierta y clase
print("Distribución de cubiertas por clase:")
print(pd.crosstab(df['Pclass'], letra_inicial, dropna=False))

print("\nSí hay relación:")
print("- Primera clase: cubiertas A, B, C, D, E (superiores)")
print("- Segunda clase: cubiertas D, E, F (intermedias)")
print("- Tercera clase: cubiertas E, F, G (inferiores)")
print("Las clases altas ocupaban cubiertas superiores, más cerca de los botes salvavidas.")
print("="*70 + "\n")

# Ejercicio 06. Detección de duplicados
# Identificación de duplicados completos y parciales (Name + Ticket)
print("="*70)
filas_duplicadas = df.duplicated().sum()
print(f"Hay {filas_duplicadas} filas completamente duplicadas.")

duplicados_parciales = df.duplicated(subset=["Name", "Ticket"]).sum()
print(f"Hay {duplicados_parciales} filas parcialmente duplicadas(nombre y ticket).")

if duplicados_parciales > 0:
    print("\nRegistros duplicados:")
    print(df[df.duplicated(subset=['Name', 'Ticket'], keep=False)][['Name', 'Ticket']])
    print("\nGestión: Revisar manualmente, podrían ser errores de registro o familiares con mismo ticket.")

print("="*70 + "\n")

# Ejercicio 07. Consistencia de datos
# Validación de rangos y detección de outliers mediante método IQR
print("="*70)

# Verificación Pclass
print("Verificación Pclass:")
pclass_valido = df['Pclass'].isin([1, 2, 3]).all()
print(f"¿Solo valores 1, 2, 3?: {pclass_valido}")
if not pclass_valido:
    print(f"Valores únicos encontrados: {df['Pclass'].unique()}")

# Verificación edades negativas
print("\nVerificación Age:")
edades_negativas = (df['Age'] < 0).sum()
print(f"Edades negativas: {edades_negativas}")

# Verificación tarifas negativas
print("\nVerificación Fare:")
tarifas_negativas = (df['Fare'] < 0).sum()
print(f"Tarifas negativas: {tarifas_negativas}")

# Outliers en tarifa mediante método IQR (Q3 + 3*IQR)
print("\nOutliers en Fare:")
q1 = df['Fare'].quantile(0.25)
q3 = df['Fare'].quantile(0.75)
iqr = q3 - q1
limite_superior = q3 + 3 * iqr
outliers = df[df['Fare'] > limite_superior]
print(f"Outliers extremos (>Q3 + 3*IQR): {len(outliers)}")
print(f"Tarifa máxima: {df['Fare'].max():.2f} (media: {df['Fare'].mean():.2f})")

if len(outliers) > 0:
    print(f"Se detectaron {len(outliers)} outliers extremos.")
    print(f"Las tarifas más altas son: {sorted(outliers['Fare'].values, reverse=True)[:5]}")

print("="*70 + "\n")

             Valores_faltantes  Porcentaje Tipo_dato
Cabin                      687       77.10       str
Age                        177       19.87   float64
Embarked                     2        0.22       str
PassengerId                  0        0.00     int64
Name                         0        0.00       str
Pclass                       0        0.00     int64
Survived                     0        0.00     int64
Sex                          0        0.00       str
Parch                        0        0.00     int64
SibSp                        0        0.00     int64
Fare                         0        0.00   float64
Ticket                       0        0.00       str

Distribución de cubiertas por clase:
Cabin    A   B   C   D   E  F  G  T  NaN
Pclass                                  
1       15  47  59  29  25  0  0  1   40
2        0   0   0   4   4  8  0  0  168
3        0   0   0   0   3  5  4  0  479

Sí hay relación:
- Primera clase: cubiertas A, B, C, D, E (superior

## Bloque 3: Tratamiento de valores faltantes

In [ ]:
# BLOQUE 3: TRATAMIENTO DE VALORES FALTANTES

# Ejercicio 08. Imputación de Age
# Imputación estratificada usando mediana por título (Mr, Mrs, Miss, Master)
print("="*70)
valores_faltantes_age = df['Age'].isna().sum()
print(f"Valores faltantes en Age: {valores_faltantes_age}")

# Extraer título del nombre mediante regex
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

# Imputar con mediana por título usando groupby().transform()
df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))

print(f"Se imputaron {valores_faltantes_age} valores usando la mediana por título.")

print("\nJustificación: La mediana es mejor que la media porque:")
print("- Es robusta a outliers (no se ve afectada por edades extremas)")
print("- La distribución de edad tiene sesgo, no es simétrica")
print("- Diferentes títulos tienen rangos de edad muy diferentes (Master vs Mrs)")

print("="*70 + "\n")

# Ejercicio 09. Decisión sobre Cabin
# Análisis de viabilidad de imputación y creación de variable binaria HasCabin
print("="*70)
missing_cabin = df['Cabin'].isna().sum()
porcentaje_missing = (missing_cabin / len(df)) * 100
print(f"Valores faltantes en Cabin: {missing_cabin} ({porcentaje_missing:.1f}%)")

print("\n¿Tiene sentido imputar? NO. Con 77% de datos faltantes, cualquier")
print("imputación sería pura especulación sin fundamento estadístico.")

print("\nAlternativa (a): Crear variable binaria HasCabin")
df['HasCabin'] = df['Cabin'].notna().astype(int)
print(f"HasCabin creada: {df['HasCabin'].sum()} tienen cabina, {(~df['HasCabin'].astype(bool)).sum()} no.")

print("\nAlternativa (b): Eliminar la columna Cabin")
print("Se perdería información sobre la cubierta (A, B, C...), pero con tanto missing no es útil.")

print("\nDecisión: Crear HasCabin.")
print("Razón: Tener/no tener cabina registrada puede ser predictor de supervivencia")
print("(pasajeros con cabina probablemente clase alta → más cerca de botes).")

print("="*70 + "\n")

# Ejercicio 10. Imputación de Embarked
# Imputación basada en análisis de pasajeros similares (clase + tarifa)
print("="*70)
valores_faltantes_embarked = df["Embarked"].isna().sum()
print(f"Hay {valores_faltantes_embarked} valores faltantes en Embarked")

# Investigar características de pasajeros con Embarked faltante
pasajeros_faltantes = df[df['Embarked'].isna()]
print("\nPasajeros con Embarked faltante:")
print(pasajeros_faltantes[['Pclass', 'Fare', 'Cabin', 'Embarked']])

# Buscar pasajeros con características similares (Pclass=1, Fare~80)
pasajeros_similares = df[(df['Pclass'] == 1) & (df['Fare'] > 79) & (df['Fare'] < 81)]
print("\nPasajeros similares (Primera clase, Fare ~80):")
print(pasajeros_similares['Embarked'].value_counts())

# Imputar con el valor más probable identificado
df.fillna({'Embarked': 'C'}, inplace=True)
print(f"\nSe imputaron los {valores_faltantes_embarked} valores con 'C' (Cherbourg)")
print("Justificación: Comparten clase, tarifa similar y cabina contigua con otros de Cherbourg")

print("="*70 + "\n")

Valores faltantes en Age: 177
Se imputaron 177 valores usando la mediana por título.

Justificación: La mediana es mejor que la media porque:
- Es robusta a outliers (no se ve afectada por edades extremas)
- La distribución de edad tiene sesgo, no es simétrica
- Diferentes títulos tienen rangos de edad muy diferentes (Master vs Mrs)

Valores faltantes en Cabin: 687 (77.1%)

¿Tiene sentido imputar? NO. Con 77% de datos faltantes, cualquier
imputación sería pura especulación sin fundamento estadístico.

Alternativa (a): Crear variable binaria HasCabin
HasCabin creada: 204 tienen cabina, 687 no.

Alternativa (b): Eliminar la columna Cabin
Se perdería información sobre la cubierta (A, B, C...), pero con tanto missing no es útil.

Decisión: Crear HasCabin.
Razón: Tener/no tener cabina registrada puede ser predictor de supervivencia
(pasajeros con cabina probablemente clase alta → más cerca de botes).

Hay 2 valores faltantes en Embarked

Pasajeros con Embarked faltante:
     Pclass  Fare Ca

## Bloque 4: Análisis exploratorio de supervivencia

In [ ]:
# BLOQUE 4: ANÁLISIS EXPLORATORIO DE SUPERVIVENCIA

# Ejercicio 11. Supervivencia por sexo
# Validación del principio "mujeres y niños primero"
print("="*70)
print("Tasa de supervivencia por sexo:")
supervivencia_sexo = df.groupby('Sex')['Survived'].mean() * 100
print(supervivencia_sexo)

print(f"\nMujeres: {supervivencia_sexo['female']:.2f}%")
print(f"Hombres: {supervivencia_sexo['male']:.2f}%")

print("\n¿Confirma 'mujeres y niños primero'? SÍ.")
print("Las mujeres tuvieron ~74% de supervivencia vs ~19% en hombres.")

print("="*70 + "\n")

# Ejercicio 12. Supervivencia por clase
# Análisis de disparidad socioeconómica en supervivencia
print("="*70)
print("Tasa de supervivencia por clase de billete:")
supervivencia_clase = df.groupby('Pclass')['Survived'].mean() * 100
print(supervivencia_clase)

print(f"\n1ª clase: {supervivencia_clase[1]:.2f}%")
print(f"2ª clase: {supervivencia_clase[2]:.2f}%")
print(f"3ª clase: {supervivencia_clase[3]:.2f}%")

print("\n¿Hay diferencias significativas? SÍ.")
print("La 1ª clase tuvo 63% de supervivencia vs 24% en 3ª clase (casi 3x más).")

print("\nHipótesis sobre ubicación:")
print("- 1ª clase: cubiertas superiores (A, B, C), acceso directo a botes salvavidas")
print("- 3ª clase: cubiertas inferiores (E, F, G), más lejos de cubierta y con barreras")

print("="*70 + "\n")

# Ejercicio 13. Tabla cruzada sexo-clase
# Análisis multivariado de supervivencia (sexo × clase)
print("="*70)
tabla_sexo_clase = (pd.crosstab(df['Sex'], df['Pclass'], values=df['Survived'], aggfunc='mean') * 100).round(2)
print("Porcentaje de supervivencia respecto al sexo y clase:")
print(tabla_sexo_clase.to_string(float_format='%.2f%%'))

print("Tenía más probabilidades de sobrevivir una mujer de tercera que un hombre de primera")
print("="*70 + "\n")

# Ejercicio 14. Efecto de la edad
# Segmentación por grupos etarios y validación del protocolo de evacuación
print("="*70)
bins = [0, 12, 17, 59, 100]
labels = ["niño", "adolescente", "adulto", "mayor"]
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

print("Tasa de supervivencia por grupo de edad:")
efecto_edad = df.groupby('AgeGroup')['Survived'].mean() * 100
print(efecto_edad)

print(f"\nNiños: {efecto_edad['niño']:.2f}%")
print(f"Adolescentes: {efecto_edad['adolescente']:.2f}%")
print(f"Adultos: {efecto_edad['adulto']:.2f}%")
print(f"Mayores: {efecto_edad['mayor']:.2f}%")

print("\n¿Los niños tuvieron ventaja? SÍ.")
print(f"Los niños tuvieron {efecto_edad['niño']:.1f}% de supervivencia, muy superior")
print("al resto de grupos, confirmando el principio 'mujeres y niños primero'.")

print("="*70 + "\n")

# Ejercicio 15. Efecto del tamaño familiar
# Análisis de supervivencia según estructura familiar a bordo
print("="*70)
df['FamilySize'] = df["SibSp"] + df["Parch"] + 1

# Categorización: solo (1), pequeña (2-4), grande (5+)
bins = [0, 1, 4, 20]
labels = ["solo", "pequeña", "grande"]
df['FamilySizeGroup'] = pd.cut(df['FamilySize'], bins=bins, labels=labels, right=True)

print("Tasa de supervivencia por tamaño familiar:")
efecto_tamanio_familiar = df.groupby('FamilySizeGroup')['Survived'].mean() * 100
print(efecto_tamanio_familiar)

print(f"\nSolo: {efecto_tamanio_familiar['solo']:.2f}%")
print(f"Familia pequeña: {efecto_tamanio_familiar['pequeña']:.2f}%")
print(f"Familia grande: {efecto_tamanio_familiar['grande']:.2f}%")

print("\n¿Las personas solas sobrevivieron más o menos?")
print("Las personas solas tuvieron MENOS supervivencia que familias pequeñas,")
print("pero MÁS que familias grandes. Las familias pequeñas (2-4) tuvieron ventaja.")

print("="*70 + "\n")

Tasa de supervivencia por sexo:
Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64

Mujeres: 74.20%
Hombres: 18.89%

¿Confirma 'mujeres y niños primero'? SÍ.
Las mujeres tuvieron ~74% de supervivencia vs ~19% en hombres.

Tasa de supervivencia por clase de billete:
Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64

1ª clase: 62.96%
2ª clase: 47.28%
3ª clase: 24.24%

¿Hay diferencias significativas? SÍ.
La 1ª clase tuvo 63% de supervivencia vs 24% en 3ª clase (casi 3x más).

Hipótesis sobre ubicación:
- 1ª clase: cubiertas superiores (A, B, C), acceso directo a botes salvavidas
- 3ª clase: cubiertas inferiores (E, F, G), más lejos de cubierta y con barreras

Porcentaje de supervivencia respecto al sexo y clase:
Pclass      1      2      3
Sex                        
female 96.81% 92.11% 50.00%
male   36.89% 15.74% 13.54%
Tenía más probabilidades de sobrevivir una mujer de tercera que un hombre de primera

Tasa de supervivencia por

## Bloque 5: Ejercicio integrador

In [8]:
def limpiar_titanic(df):
    df = df.copy()
    
    # Extraer título del nombre
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
    
    # Imputar Age con mediana por título
    df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
    
    # Crear variable HasCabin
    df['HasCabin'] = df['Cabin'].notna().astype(int)
    
    # Imputar Embarked con el valor más frecuente
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    
    # Crear FamilySize
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # Crear AgeGroup
    bins = [0, 12, 17, 59, 120]
    labels = ['niño', 'adolescente', 'adulto', 'mayor']
    df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
    
    # Eliminar columnas innecesarias
    df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
    
    return df

# Recargar dataset limpio
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df_original = pd.read_csv(url)

# Aplicar función
df_limpio = limpiar_titanic(df_original)

# Verificar
print(f"Shape original: {df_original.shape}")
print(f"Shape limpio: {df_limpio.shape}")
print(f"\nColumnas eliminadas: {set(df_original.columns) - set(df_limpio.columns)}")
print(f"Columnas nuevas: {set(df_limpio.columns) - set(df_original.columns)}")
print(f"\nValores faltantes en Age: {df_limpio['Age'].isna().sum()}")
print(f"Valores faltantes en Embarked: {df_limpio['Embarked'].isna().sum()}")

Shape original: (891, 12)
Shape limpio: (891, 12)

Columnas eliminadas: {'Cabin', 'Name', 'PassengerId', 'Ticket'}
Columnas nuevas: {'FamilySize', 'Title', 'HasCabin', 'AgeGroup'}

Valores faltantes en Age: 0
Valores faltantes en Embarked: 0
